### Import packages

In [ ]:
import os
from aicsimageio import AICSImage
from skimage import io
import re

### Specify paths

In [ ]:
# Specify path to root folder
root_folder = 'D:/Data/2023-07-CarolineAsselman-FIm/Analysis_v2'

# Path to input folder
images_path = os.path.join(root_folder, 'images')

# Path to output folder
corrected_images_path = os.path.join(root_folder, 'images_size_corrected')
os.makedirs(corrected_images_path, exist_ok = True)

### Get maximum image XY-shifts for all ROIs

In [ ]:
def get_shifts_per_ROI(data_folder):
    image_shifts_per_ROI = {}
    max_shifts_per_ROI = {}

    for ROI in os.listdir(data_folder):
        ROI_image_shifts = []
        ROI_max_shift_x = float('-inf')  # Initialize to negative infinity
        ROI_max_shift_y = float('-inf')  # Initialize to negative infinity

        ROI_path = os.path.join(data_folder, ROI)

        for file in os.listdir(ROI_path):
            if file.endswith('.tif'):
                img = AICSImage(os.path.join(ROI_path, file))

                image_shifts = (int(re.findall(r'PositionX="(\d+)"', img.metadata)[0]), int(re.findall(r'PositionY="(\d+)"', img.metadata)[0]))

                ROI_image_shifts.append(image_shifts)
                ROI_max_shift_x = max(ROI_max_shift_x, image_shifts[0])
                ROI_max_shift_y = max(ROI_max_shift_y, image_shifts[1])

        image_shifts_per_ROI[ROI] = ROI_image_shifts
        max_shifts_per_ROI[ROI] = (ROI_max_shift_x, ROI_max_shift_y)
        
    for ROI, shifts in image_shifts_per_ROI.items():
        print(f"ROI: {ROI}")
        print(f"Unique images shifts: {set(shifts)}")

    for ROI, max_shifts in max_shifts_per_ROI.items():
        print(f"Folder: {ROI}")
        print(f"Maximum shifts: {max_shifts}")

    return image_shifts_per_ROI, max_shifts_per_ROI

In [ ]:
image_shifts_per_ROI, max_shifts_per_ROI = get_shifts_per_ROI(data_folder = images_path)

### Correct image shifts

In [ ]:
def correct_shifts(data_folder, output_folder, max_shifts_per_ROI):
    for ROI, max_shifts in max_shifts_per_ROI.items():
        output_subfolder = os.path.join(output_folder, ROI)
        os.makedirs(output_subfolder, exist_ok=True)
        
        for file in os.listdir(os.path.join(data_folder, ROI)):
            if file.endswith('.tif'):
                img = AICSImage(os.path.join(data_folder, ROI, file))
                img_xy = img.data[0,0,0,:,:]
                
                image_shifts = (int(re.findall(r'PositionX="(\d+)"', img.metadata)[0]), int(re.findall(r'PositionY="(\d+)"', img.metadata)[0]))
                shift_correction = (max_shifts[0] - image_shifts[0], max_shifts[1] - image_shifts[1])
                
                img_corrected = img_xy[shift_correction[1]:, shift_correction[0]:] 
                
                output_file = os.path.join(output_subfolder, file)
                io.imsave(output_file, img_corrected)               

In [ ]:
correct_shifts(data_folder = images_path, output_folder = corrected_images_path, max_shifts_per_ROI = max_shifts_per_ROI)

### Apply an additional manual shift to correct for inaccurate alignment from MACS IQ View preprocessing

In [ ]:
def manual_correction(data_folder, output_folder, shifted_channels_per_ROI, manual_correction_per_ROI):
    for ROI, manual_correction in manual_correction_per_ROI.items():
        output_subfolder = os.path.join(output_folder, ROI)
        os.makedirs(output_subfolder, exist_ok=True)
        
        for file in os.listdir(os.path.join(data_folder, ROI)):
            if file.endswith('.tif') and not any(channel in file for channel in shifted_channels_per_ROI[ROI]):
                img = io.imread(os.path.join(data_folder, ROI, file))
                img_corrected = img[manual_correction[1]:, manual_correction[0]:] 
                
                output_file = os.path.join(output_subfolder, file)
                io.imsave(output_file, img_corrected)   

In [ ]:
shifted_channels_per_ROI = {
    'A1_ROI1' : [
        'CD45RA_C-REAL164',
        'AnnexinI_C-REA1122',
        'MLC2v_C-REA401',
        'CD279_C-REA116',
        'Cytokeratin1013_C-REA1138',
        'CD196_C-REA190',
        'CD90_C-REAL677',
        'Metallothionein_C-REA1139',
    ],
}

manual_correction_per_ROI = {
    'A1_ROI1' : (15,0),

}

manual_correction(corrected_images_path, corrected_images_path, shifted_channels_per_ROI, manual_correction_per_ROI)

### Get minimum XY-dimensions for all ROIs

In [ ]:
def get_dimensions_per_ROI(data_folder):
    image_dimensions_per_ROI = {}
    min_dimensions_per_ROI = {}

    for ROI in os.listdir(data_folder):
        ROI_image_dimensions = []
        ROI_min_x = float('inf')  # Initialize to positive infinity
        ROI_min_y = float('inf')  # Initialize to positive infinity

        ROI_path = os.path.join(data_folder, ROI)

        for file in os.listdir(ROI_path):
            if file.endswith('.tif'):
                img = AICSImage(os.path.join(ROI_path, file))

                image_shape = (img.dims.X, img.dims.Y)

                ROI_image_dimensions.append(image_shape)
                ROI_min_x = min(ROI_min_x, image_shape[0])
                ROI_min_y = min(ROI_min_y, image_shape[1])

        image_dimensions_per_ROI[ROI] = ROI_image_dimensions
        min_dimensions_per_ROI[ROI] = (ROI_min_x, ROI_min_y)

    for ROI, dimensions in image_dimensions_per_ROI.items():
        print(f"ROI: {ROI}")
        print(f"Unique images sizes: {set(dimensions)}")

    for ROI, min_dims in min_dimensions_per_ROI.items():
        print(f"Folder: {ROI}")
        print(f"Minimum dimensions: {min_dims}")
        
    return image_dimensions_per_ROI, min_dimensions_per_ROI

In [ ]:
image_dimensions_per_ROI, min_dimensions_per_ROI = get_dimensions_per_ROI(data_folder = corrected_images_path)

### Correct image sizes

In [ ]:
def correct_sizes(data_folder, output_folder, min_dimensions_per_ROI):
    for ROI, min_dims in min_dimensions_per_ROI.items():
        output_subfolder = os.path.join(output_folder, ROI)
        os.makedirs(output_subfolder, exist_ok=True)
        
        for file in os.listdir(os.path.join(data_folder, ROI)):
            if file.endswith('.tif'):
                img = io.imread(os.path.join(data_folder, ROI, file))
                img_corrected = img[:min_dims[1], :min_dims[0]]
                
                output_file = os.path.join(output_subfolder, file)
                io.imsave(output_file, img_corrected)   

In [ ]:
correct_sizes(data_folder = corrected_images_path, output_folder = corrected_images_path, min_dimensions_per_ROI = min_dimensions_per_ROI)

### Check image sizes

In [ ]:
image_dimensions_per_ROI, min_dimensions_per_ROI = get_dimensions_per_ROI(data_folder=corrected_images_path)